# The imports

In [1]:
import zarr
import xarray as xr
import time
import dask

# The workers

In [2]:
ask_workers=224
memory='60GB'

In [3]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=24,name='pangeo',walltime='00:30:00',
                       job_extra=['--constraint=BDW28','--exclusive',
                                  '--nodes=1'],memory=memory,
                       interface='ib0') 
cluster.scale(ask_workers)
c= Client(cluster)
c



/scratch/cnt0024/hmg2840/albert7a/DEV/git/conda-pack/perf-pangeo/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34486 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://172.30.100.3:38838 Dashboard: http://172.30.100.3:34486/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


Workers= 224, Cores=448, Memory=479.36 GB


# The data

In [6]:
%time ds=xr.open_zarr('/store/albert7a/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')

CPU times: user 86.9 ms, sys: 35.4 ms, total: 122 ms
Wall time: 1.09 s


In [7]:
%time mean=ds.sossheig.mean(dim='time_counter')

CPU times: user 196 ms, sys: 23.2 ms, total: 219 ms
Wall time: 357 ms


In [8]:
%time mean.load()

CPU times: user 39.6 s, sys: 2.86 s, total: 42.5 s
Wall time: 51.5 s


<xarray.DataArray 'sossheig' (y: 4729, x: 8354)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Dimensions without coordinates: y, x

In [9]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
